In [1]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

 
client_d500954240e145ad83c127ad33a1da87 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='',
    ibm_auth_endpoint=" ",
    config=Config(signature_version=''),
    endpoint_url='')

#Loading of the data file into dataframe

body = client_d500954240e145ad83c127ad33a1da87.get_object(Bucket='capstonefinalprojectbattleofthene-donotdelete-pr-chfvhtlsg8gil5',Key='mum-final-new.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()




 

,City,Pincode,Neighborhood,Latitude,Longitude
0,Mumbai,400001,Stock Exchange,18.6291,72.8919
1,Mumbai,400001,M.P.T.,18.6291,72.8919
2,Mumbai,400002,Kalbadevi,17.0509,73.2910
3,Mumbai,400002,S. C. Court,17.0509,73.2910
4,Mumbai,400002,Ramwadi,17.0509,73.2910


In [2]:

import numpy as np

# Section 1: Explore & Cluster the neighbourhoods in Mumbai
# Segmentation & Clustering to be done for Mumbai  

!pip install folium
from geopy.geocoders import Nominatim
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium




     |████████████████████████████████| 94 kB 6.3 MB/s  eta 0:00:01


In [3]:
#Loading of credentials for Foursquare API

CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = ''



df_data_1.loc[0, 'Neighborhood']

'Stock Exchange'

In [4]:


neighborhood_latitude = df_data_1.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_data_1.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_data_1.loc[0, 'Neighborhood'] # neighborhood name

In [5]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=UFFQW0IYMUZYESSNIJDL5ZGAVIPXMXPKFZF2XRMGL3QGLSTX&client_secret=5DKIRDX5P5LDXFZIRHQHHOK5BLIYNKULKMEQUVZ4VQF2VBQA&v=20210108&ll=18.6291,72.8919&radius=500&limit=100'

In [ ]:
#Use of Foursquare API to get nearby venues and categories information for Mumbai neighborhoods  

In [6]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
mumbai_data_venues = getNearbyVenues(names=df_data_1['Neighborhood'],
                                   latitudes=df_data_1['Latitude'],
                                   longitudes=df_data_1['Longitude']
                                  )

Stock Exchange
M.P.T.
Kalbadevi
S. C. Court
Ramwadi
Thakurdwar
Mandvi (Mumbai)
Null Bazar
B.P.Lane
Masjid
Girgaon
Madhavbaug
Colaba
Malabar Hill
Tardeo
Falkland Road
Mumbai Central
M A Marg
J.J.Hospital
Kamathipura
Noor Baug
Princess Dock
Chinchbunder
Mazgaon
Mazgaon Road
V K Bhavan
Mazgaon Dock
Dockyard Road
Agripada
Chinchpokli
Jacob Circle
Haines Road
Parel
Chamarbaug
Haffkin Institute
Lal Baug
Parel Naka
Parel Rly Work Shop
BEST STaff Quarters
Delisle Road
Naigaon (Mumbai)
Dadar
Dadar Colony
Sewri
Mahim
Mahim Bazar
Mori Road
Kapad Bazar
Mahim East
Dharavi Road
Dharavi
Worli
Worli Naka
Matunga
Marine Lines
Churchgate
Central Building
Nariman Point
New Yogakshema
Sion
Raoli Camp
Transit Camp
Chunabhatti
Nehru Nagar (Mumbai)
Prabhadevi
New Prabhadevi Road
Cumballa Sea Face
Gowalia Tank
Dr Deshmukh Marg
Cumballa Hill
V J B Udyan
Ranade Road
Gokhale Road (Mumbai)
Bhawani Shankar
Bhawani Shankar Rd
S V S Marg
Shivaji Park (Mumbai)
Santacruz P&t Colony
A I Staff Colony
Worli Police Camp
C

In [9]:

print(mumbai_data_venues.shape)
mumbai_data_venues.head()

(1884, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Stock Exchange,18.6291,72.8919,ACHI BIZ SERVICES PTE. LTD.,18.630000,72.891530,Business Service
1,Stock Exchange,18.6291,72.8919,DigiComfy,18.629715,72.889958,Business Service
2,Stock Exchange,18.6291,72.8919,Kadam Holidays,18.629150,72.888194,Hotel
3,M.P.T.,18.6291,72.8919,ACHI BIZ SERVICES PTE. LTD.,18.630000,72.891530,Business Service
4,M.P.T.,18.6291,72.8919,DigiComfy,18.629715,72.889958,Business Service


In [10]:
mumbai_data_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
A I Staff Colony,7,7,7,7,7,7
Aareymilk Colony,7,7,7,7,7,7
Agripada,4,4,4,4,4,4
Airport (Mumbai),4,4,4,4,4,4
Andheri,7,7,7,7,7,7
...,...,...,...,...,...,...
Worli,6,6,6,6,6,6
Worli Colony,15,15,15,15,15,15
Worli Naka,6,6,6,6,6,6


In [11]:
print('There are {} uniques categories.'.format(len(mumbai_data_venues['Venue Category'].unique())))

There are 111 uniques categories.


In [12]:
# one hot encoding
mumbai_onehot = pd.get_dummies(mumbai_data_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mumbai_onehot['Neighborhood'] = mumbai_data_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[fixed_columns]

mumbai_onehot.head()

,Neighborhood,American Restaurant,Aquarium,Asian Restaurant,Astrologer,Athletics & Sports,Bakery,Bank,Bar,Beach,...,Tea Room,Tennis Court,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Wine Shop,Women's Store
0,Stock Exchange,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Stock Exchange,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Stock Exchange,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M.P.T.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M.P.T.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
mumbai_onehot.shape

(1884, 112)

In [14]:
mumbai_grouped = mumbai_onehot.groupby('Neighborhood').mean().reset_index()
mumbai_grouped.head()

,Neighborhood,American Restaurant,Aquarium,Asian Restaurant,Astrologer,Athletics & Sports,Bakery,Bank,Bar,Beach,...,Tea Room,Tennis Court,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Wine Shop,Women's Store
0,A I Staff Colony,0.0,0.0,0.142857,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
1,Aareymilk Colony,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.142857,0.0
2,Agripada,0.0,0.0,0.000000,0.0,0.000000,0.250000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
3,Airport (Mumbai),0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
4,Andheri,0.0,0.0,0.000000,0.0,0.142857,0.142857,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.000000,0.0


In [15]:
mumbai_grouped.shape

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [16]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = mumbai_grouped['Neighborhood']

for ind in np.arange(mumbai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mumbai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A I Staff Colony,Furniture / Home Store,Café,Asian Restaurant,Spa,Indian Restaurant,Coffee Shop,Modern European Restaurant,Women's Store,Flea Market,Fish & Chips Shop
1,Aareymilk Colony,Indian Restaurant,Smoke Shop,Fast Food Restaurant,Wine Shop,Gym / Fitness Center,Bookstore,Food,Flower Shop,Flea Market,Fish & Chips Shop
2,Agripada,History Museum,Bakery,Multiplex,Fast Food Restaurant,Women's Store,Food & Drink Shop,Diner,Donut Shop,Electronics Store,Event Space
3,Airport (Mumbai),Dessert Shop,Italian Restaurant,Gym,Café,Food & Drink Shop,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market
4,Andheri,Bus Station,Bike Rental / Bike Share,Gym,Train Station,Athletics & Sports,Bakery,Pizza Place,Flower Shop,Flea Market,Fish & Chips Shop
...,...,...,...,...,...,...,...,...,...,...,...
192,Worli,Café,Nightclub,French Restaurant,Scenic Lookout,Fish & Chips Shop,Women's Store,Flower Shop,Dessert Shop,Diner,Donut Shop
193,Worli Colony,Coffee Shop,Indian Restaurant,Café,Lounge,Bus Station,Gym,Vegetarian / Vegan Restaurant,Convenience Store,Juice Bar,Farmers Market
194,Worli Naka,Café,Nightclub,French Restaurant,Scenic Lookout,Fish & Chips Shop,Women's Store,Flower Shop,Dessert Shop,Diner,Donut Shop
195,Worli Police Camp,Coffee Shop,Indian Restaurant,Café,Lounge,Bus Station,Gym,Vegetarian / Vegan Restaurant,Convenience Store,Juice Bar,Farmers Market


In [17]:
# set number of clusters
kclusters = 3

mumbai_grouped_clustering = mumbai_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=int32)

In [18]:
mumbai_merged = df_data_1
mumbai_merged=mumbai_merged[6:]

print(len(mumbai_merged))
print(len(neighborhoods_venues_sorted))



211
197


In [22]:



# add clustering labels
neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_


#Cluster1
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 0, neighborhoods_venues_sorted.columns[[1] + list(range(5, neighborhoods_venues_sorted.shape[1]))]]


,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Furniture / Home Store,Indian Restaurant,Coffee Shop,Modern European Restaurant,Women's Store,Flea Market,Fish & Chips Shop,0
1,Indian Restaurant,Gym / Fitness Center,Bookstore,Food,Flower Shop,Flea Market,Fish & Chips Shop,0
2,History Museum,Women's Store,Food & Drink Shop,Diner,Donut Shop,Electronics Store,Event Space,0
3,Dessert Shop,Food & Drink Shop,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,0
4,Bus Station,Athletics & Sports,Bakery,Pizza Place,Flower Shop,Flea Market,Fish & Chips Shop,0
...,...,...,...,...,...,...,...,...
192,Café,Fish & Chips Shop,Women's Store,Flower Shop,Dessert Shop,Diner,Donut Shop,0
193,Coffee Shop,Bus Station,Gym,Vegetarian / Vegan Restaurant,Convenience Store,Juice Bar,Farmers Market,0
194,Café,Fish & Chips Shop,Women's Store,Flower Shop,Dessert Shop,Diner,Donut Shop,0
195,Coffee Shop,Bus Station,Gym,Vegetarian / Vegan Restaurant,Convenience Store,Juice Bar,Farmers Market,0


In [23]:
#Cluster2
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 1, neighborhoods_venues_sorted.columns[[1] + list(range(5, neighborhoods_venues_sorted.shape[1]))]]


,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
6,Business Service,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,1
16,Business Service,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,1
37,Business Service,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,1
56,Business Service,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,1
64,Business Service,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,1
68,Business Service,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,1
71,Business Service,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,1
72,Business Service,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,1
78,Business Service,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,1
93,Business Service,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,1


In [25]:
#Cluster3
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 2, neighborhoods_venues_sorted.columns[[1] + list(range(5, neighborhoods_venues_sorted.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
14,Clothing Store,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,2
18,Clothing Store,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,2
20,Clothing Store,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,2
36,Clothing Store,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,2
57,Clothing Store,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,2
81,Clothing Store,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,2
113,Clothing Store,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,2
118,Clothing Store,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,2
149,Clothing Store,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,2
154,Clothing Store,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,2
